In [252]:
import pandas as pd
import sqlite3

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

#models
from sklearn.naive_bayes import MultinomialNB
import xgboost as xgb

#success metric
from  sklearn.metrics  import accuracy_score

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_colwidth', -1)

In [194]:
# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect('corpus.sqlite3')
df_annots = pd.read_sql_query("SELECT * FROM Annotations", con)

# Verify that result of SQL query is stored in the dataframe
print(df.head())

con.close()

         Date   Sym  Data2  Data3  Data4
0  2015-05-08  aapl  11     5      55   
1  2015-05-07  aapl  8      8      108  
2  2015-05-06  aapl  10     6      66   
3  2015-05-05  aapl  15     1      121  
4  2015-05-08  aaww  110    50     55   


In [195]:
#checking the shape of loaded df
df_annots.shape

(58568, 4)

In [196]:
#checking the category data, apart from sentiment there are other categories
df_annots['Category'].value_counts()

PersonalStories      11336
PossiblyFeedback     8039 
SentimentNeutral     5599 
OffTopic             5599 
Discriminating       5599 
SentimentPositive    5599 
ArgumentsUsed        5599 
SentimentNegative    5599 
Inappropriate        5599 
Name: Category, dtype: int64

In [197]:
#we are interested in sentiment data
#create separate df for neutral, positive and negative sentiments
df_neutral = df_annots[df_annots['Category']=='SentimentNeutral']
df_neutral.shape

(5599, 4)

In [198]:
df_positive = df_annots[df_annots['Category']=='SentimentPositive']
df_positive.shape

(5599, 4)

In [199]:
df_negative = df_annots[df_annots['Category']=='SentimentNegative']
df_negative.shape

(5599, 4)

In [200]:
#merging all three above df's to one df_sent
df_sent = pd.concat([df_neutral, df_positive, df_negative])
df_sent.shape

(16797, 4)

In [201]:
#checking the number of unique values of ID_Posts
df_sent['ID_Post'].nunique()

3599

In [202]:
#according to descriptions from SQL database only posts marked with 1 in "value" column are indeed neutral, pos or neg
#therefore I delete rows with 0 in 'value' column
df_final_sent = df_sent[df_sent['Value']==1]

In [203]:
#checking final shape
df_final_sent.shape

(5599, 4)

In [204]:
#checking the number of annotators, we have 4 different annotators
df_final_sent['ID_Annotator'].value_counts()

1    2594
2    1513
3    1000
4    492 
Name: ID_Annotator, dtype: int64

In [205]:
#we have 3599 unique posts and 5599 rows in the df, which means that some posts have been annotated multiple times
df_final_sent.head()

,ID_Post,ID_Annotator,Category,Value
7,3326,1,SentimentNeutral,1
16,3326,2,SentimentNeutral,1
25,3326,3,SentimentNeutral,1
52,5321,3,SentimentNeutral,1
61,5590,1,SentimentNeutral,1


In [206]:
#delete the duplicates where Post_ID and Category values are the same
df_final_sent.drop_duplicates(subset=['ID_Post','Category'], inplace=True)

In [207]:
#after droping duplicates we still have 4034 rows, which suggests that some post have been annotated with different classes
df_final_sent.shape

(4034, 4)

In [132]:
#indeed some post still have 2 or 3 different annotations
df_final_sent['ID_Post'].value_counts()

630070    3
413265    3
573885    3
671191    3
510887    3
180020    3
657483    3
227362    3
924705    3
110246    3
302266    2
260676    2
694673    2
203025    2
956972    2
246926    2
792588    2
509250    2
438833    2
493463    2
635164    2
333220    2
676645    2
806132    2
620098    2
707126    2
655951    2
14677     2
416422    2
258384    2
         ..
578860    1
904491    1
124202    1
904519    1
124234    1
124266    1
124250    1
875881    1
124131    1
140646    1
875877    1
876011    1
904407    1
672933    1
875873    1
124254    1
875869    1
875865    1
27979     1
832856    1
929742    1
124246    1
875861    1
140626    1
875857    1
904527    1
124238    1
875853    1
124081    1
904350    1
Name: ID_Post, Length: 3599, dtype: int64

In [208]:
#example where the same post has been annotated in three different classes (neutral, positive, negative)
df_final_sent[df_final_sent['ID_Post']==630070]

,ID_Post,ID_Annotator,Category,Value
16045,630070,1,SentimentNeutral,1
16064,630070,3,SentimentPositive,1
16053,630070,2,SentimentNegative,1


In [209]:
#I decide to remove such inconsistent annotations from dataset (the might be confusing for model)
#maybe we could use them for further testing of the model
df_final_sent['is_neutral'] = df_final_sent['Category'].map(lambda x: 1 if str(x)=='SentimentNeutral' else 0)
df_final_sent['is_positive'] = df_final_sent['Category'].map(lambda x: 1 if str(x)=='SentimentPositive' else 0)
df_final_sent['is_negative'] = df_final_sent['Category'].map(lambda x: 1 if str(x)=='SentimentNegative' else 0)

In [211]:
temp = df_final_sent.groupby('ID_Post')[['is_neutral', 'is_positive', 'is_negative']].sum()
temp.reset_index(level=None, drop=False, inplace=True, col_level=0, col_fill='')
temp['sum_agg'] = temp['is_neutral'] + temp['is_positive'] + temp['is_negative']
del temp['is_neutral']
del temp['is_positive']
del temp['is_negative']

df_all = pd.merge(df_final_sent, temp, on='ID_Post')

In [214]:
df_all.head(5)
df_all.shape

(4034, 8)

In [213]:
#separate df with vague annotations for eventual tests
df_sent_vague = df_all[df_all['sum_agg']>1]
df_sent_vague.shape

(860, 8)

In [216]:
#separate df with clear annotations for training
df_sentiment = df_all[df_all['sum_agg']==1]
df_sentiment.shape

(3174, 8)

In [224]:
#summary of training dataset
print('We have {} neutral posts in our training dataset'.format(df_sentiment['is_neutral'].sum()))
print('We have {} positive posts in our training dataset'.format(df_sentiment['is_positive'].sum()))
print('We have {} negative posts in our training dataset'.format(df_sentiment['is_negative'].sum()))

We have 1622 neutral posts in our training dataset
We have 25 positive posts in our training dataset
We have 1527 negative posts in our training dataset


In [227]:
1622/3174*100, 25/3174*100, 1527/3174*100

(51.102709514807806, 0.7876496534341524, 48.109640831758036)

In [229]:
# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect('corpus.sqlite3')
df_posts = pd.read_sql_query("SELECT * FROM Posts", con)

# Verify that result of SQL query is stored in the dataframe
print(df_posts.head())

con.close()

   ID_Post  ID_Parent_Post  ID_Article  ID_User                CreatedAt  \
0  1       NaN              1           9089     2003-04-23 14:52:41.870   
1  2       NaN              1           29367    2003-11-04 16:21:57.850   
2  3        2.0             1           5095     2004-01-28 12:57:28.240   
3  4        3.0             1           1682     2004-02-03 20:32:39.123   
4  5       NaN              1           3343     2004-03-02 11:37:44.100   

    Status                   Headline  \
0  deleted                              
1  online   Newsletter "DER STANDARD"   
2  deleted  Auch begeistert...          
3  deleted  Abmeldeprobleme             
4  online                               

                                                                                                                                                                                                                                                                                                        

In [230]:
df_posts.head()

,ID_Post,ID_Parent_Post,ID_Article,ID_User,CreatedAt,Status,Headline,Body,PositiveVotes,NegativeVotes
0,1,NaN,1,9089,2003-04-23 14:52:41.870,deleted,,,0,0
1,2,NaN,1,29367,2003-11-04 16:21:57.850,online,"Newsletter ""DER STANDARD""","Ich bin begeistert von den STANDARD - Newslettern.\r\nIn keiner anderen E-Zeitung gibt es eine solche Viel=\r\nfalt und so interessante Kommentare bzw. eine so\r\ngenaue Berichterstattung. \r\n Macht weiter so, ich finde es jedenfalls toll.\r\n\r\n\r\n Johann Radakovits",0,0
2,3,2.0,1,5095,2004-01-28 12:57:28.240,deleted,Auch begeistert...,"... Aber momentan funktioniert das Abmelden oder Adressen ändern überhaupt nicht! In USA macht \r\nman dies auf den Servern einfacher.\r\n\r\nAnsonsten ist DER STANDARD ja wirklich das Beste, was es in Österreich gibt...\r\nWeiter im ""Text""\r\n;-)carpeDiem(-;\r\n",0,0
3,4,3.0,1,1682,2004-02-03 20:32:39.123,deleted,Abmeldeprobleme,"Es ist ganz einfach nervend!\r\nVor kurzem habe ich den news-letter angemeldet in der hoffnungsvollen Gewissheit (weil vertrauenserweckend angekündigt), ihn jederzeit wieder stornieren zu können. Aber leider führt kein Weg dahin. Ich wäre dankbar für eine diesbezügliche Hilfe des Lesers dieser eMail.",0,0
4,5,NaN,1,3343,2004-03-02 11:37:44.100,online,,und sie als mitarbeiter sind natuerlich objektiv,0,0


In [231]:
df_posts.columns

Index(['ID_Post', 'ID_Parent_Post', 'ID_Article', 'ID_User', 'CreatedAt',
       'Status', 'Headline', 'Body', 'PositiveVotes', 'NegativeVotes'],
      dtype='object')

In [232]:
columns_to_delete = ['ID_Parent_Post', 'ID_Article', 'ID_User', 'CreatedAt',
       'Status', 'Headline', 'PositiveVotes', 'NegativeVotes' ]

In [233]:
#delete not needed columns
df_posts = df_posts.drop(columns_to_delete, axis = 1)

In [234]:
df_posts.sample(5)

,ID_Post,Body
205457,205458,"Na, wurdest von der Vali in Saalbach abgehängt?"
229507,229508,und kickl sein vize\r\n\r\ndie tätatn wien weiter bringen \r\n\r\n;-)
291304,291305,Die Wahrheit und die FPÖ - eine Geschichte voller Missverständnisse.
975251,975252,"Sevilla hat es einfach klug gemacht und sich vor der Saison mit Serie A Größen wie Rami, Andreolli und Llorente verstärkt."
205445,205446,"Und wo war da jetzt das Argument gegen den Feminismus? Gäbe es den nicht, hätte das Mädchen nicht mal die Möglichkeit, an einem Bewerb in dieser Sportart teilzunehmen - viel zu gefährlich für eine heranreifende Mutter.\r\n\r\nEs ist ein Irrglaube, dass der Feminismus etwas dagegen hat, wenn Frauen Hausfrauen und Mütter werden. Es sollte nur von der Gesellschaft nicht mehr gefördert werden, als wenn Sie Downhill fährt."


In [274]:
#merge df with comments with df with sentiment, delete rows with empty posts
df_final = pd.merge(df_sentiment, df_posts, on='ID_Post')
df_final = df_final.dropna(subset=['Body'])
df_final.shape

(3037, 9)

In [276]:
df_final.sample(5)

,ID_Post,ID_Annotator,Category,Value,is_neutral,is_positive,is_negative,sum_agg,Body
766,124560,1,SentimentNeutral,1,1,0,0,1,"Ich weiß, was Menschenverachtung bedeutet. Das ist die Haltung des jeweiligen Nebenmenschen, gelt. Der Vorwurf wird (inflationär) missbraucht. Beispielsweise auch hier von Ihnen. Sie unterstellen Unverständnis, weil jemand Ihre Sicht nicht teilt. Weil sie sich Ihrer Ideologie nicht fügt. Sie unterstellen Verächtlichkeit gg Bedürftige, obwohl sie sehr deutlich gg Privilegierte anschreibt. Inhaltlich bieten Sie leider nichts an. Was genau kritisieren Sie denn? Wenn ich an Ihrer Stelle wäre, würde ich mir einzelne Sätze herauspicken und ihre vermeintlich prekäre Wirkung demonstrieren. Aber Sie belassen es beim popanzigen Vorwurf der Menschenverachtung. Ich teile Ihre Meinung nicht, ohne Ihnen deshalb Menschenverachtung vorzuwerfen ;-)"
1808,942203,1,SentimentNegative,1,0,0,1,1,"""Doch ein Einreisezertifikat sei von der kenianischen Botschaft nicht ausgestellt worden.""\r\n\r\nBotschaft sofort schliessen.\r\n\r\nDie ist Mitschuld am Mord und hat ihr Recht hier zu sein verwirkt.\r\n\r\nWieso toleriert Österreich Botschaften die Gangster schicken?"
2773,876044,4,SentimentNegative,1,0,0,1,1,"Griechische Behörden beschuldigen Aktivisten, die Flüchtlinge im Lager Idomeni anzustacheln. Immer wieder gibt es dort Tumulte. Wer ist wirklich verantwortlich? Eine Spurensuche vor Ort.\r\n\r\nhttp://www.spiegel.de/politik/ausland/fluechtlinge-in-idomeni-aktivisten-sollen-unruhen-anzetteln-a-1086805.html"
2908,896137,2,SentimentNegative,1,0,0,1,1,"Ähm!? Und warum genau ist das ihrer Meinung nach kein zulässiges Argument? \r\n\r\nDie Anzahl der Asylanträge hat sich 2015 gegenüber 2014 fast verdreifacht; da ist eine entsprechende absolute Steigerung der Straftaten -ohne das jetzt sonst irgendwie bewerten zu wollen- ganz einfach eine rein statistische Konsequenz daraus. \r\nWenn wir jetzt plötzlich doppelt so viele Österreicher im Land hätten, dann würden die (absoluten) Statistiken ja auch nicht gleich bleiben, oder ?!\r\n\r\nPs: Außerdem sollten Sie auf präzisere Formulierungen achten, wenn Sie mit Statistiken hantieren; es müsste heißen ""in der Gruppe der 14-17 jährigen Asylwerber"". Ihre Formulierung suggeriert eine Verdopplung der Jugendkriminalität insgesamt in dieser Altersgruppe."
2456,140625,2,SentimentNegative,1,0,0,1,1,"Schade das sein echter Name nicht genannt wurde, da hätte die Asylbehörde einen Beweis in der Hand das er nur ein Scheinasylant ist!"


In [277]:
#maping sentiment into one column: -1 for negative, 0 for neutral, 1 for positive
sentiment_map = {'SentimentNeutral': 0, 'SentimentPositive': 1, 'SentimentNegative': -1}
df_final['sentiment'] = df_final['Category'].map(sentiment_map)

In [279]:
df_final.head(5)

,ID_Post,ID_Annotator,Category,Value,is_neutral,is_positive,is_negative,sum_agg,Body,sentiment
0,3326,1,SentimentNeutral,1,1,0,0,1,Top qualifizierte Leute verdienen auch viel.,0
1,5590,1,SentimentNeutral,1,1,0,0,1,"Sorry, aber die FPÖ tut eigentlich gar nichts und gewinnt TROTZDEM.",0
2,8213,1,SentimentNeutral,1,1,0,0,1,Na wer weis was da vorgefallen ist...,0
3,13038,1,SentimentNeutral,1,1,0,0,1,was die meise gibt kein black metal konzert?,0
4,13060,1,SentimentNeutral,1,1,0,0,1,"Musst nicht, sonst gäbs weniger Spaß! ^^",0


In [281]:
#checking the top words used in reviews by frequency distribution

#combining all reviews into one string
posts = df_final['Body'].str.cat(sep='')

#spliting text into separate words
tokenizer = RegexpTokenizer(r'\w+')
tokens = tokenizer.tokenize(posts)

#setting vocabulary
vocabulary = set(tokens)
print(len(vocabulary))

freq_distr = nltk.FreqDist(tokens)

#sorting top 50 tokens
sorted(freq_distr, key=freq_distr.__getitem__, reverse=True)[0:50]

19126


['die',
 'und',
 'der',
 'nicht',
 'ist',
 'in',
 'das',
 'zu',
 'es',
 'auch',
 'sie',
 'sich',
 'ein',
 'dass',
 'man',
 'den',
 'von',
 'ich',
 'mit',
 'sind',
 'aber',
 'eine',
 'für',
 'als',
 'so',
 'wird',
 'auf',
 'nur',
 'wie',
 'im',
 'hat',
 'Sie',
 'werden',
 'wenn',
 'oder',
 'dann',
 'haben',
 'kann',
 'noch',
 'ja',
 'schon',
 'dem',
 'was',
 'er',
 'aus',
 'bei',
 'Das',
 'Frauen',
 'an',
 'da']

In [282]:
#we have to remove stopwords

stop_words = set(stopwords.words('german'))
tokens = [w for w in tokens if not w in stop_words]
freq_distr = nltk.FreqDist(tokens)

#sorting top 50 tokens
sorted(freq_distr, key=freq_distr.__getitem__, reverse=True)[0:50]

['Sie',
 'ja',
 'schon',
 'Das',
 'Frauen',
 'Ich',
 'mehr',
 'Die',
 'gibt',
 'Und',
 'immer',
 'mal',
 'Es',
 'Menschen',
 'Wenn',
 'einfach',
 'Israel',
 'wohl',
 'wäre',
 'geht',
 'Frau',
 'Männer',
 'http',
 'Gewalt',
 'wurde',
 'Was',
 'gut',
 'www',
 'ganz',
 's',
 'Aber',
 '2',
 'Der',
 'wirklich',
 'halt',
 'Artikel',
 'Österreich',
 'Mann',
 'viele',
 'Land',
 'In',
 'gar',
 'kommt',
 'genau',
 'müssen',
 'hätte',
 'tun',
 'Flüchtlinge',
 '1',
 'weniger']

NAIVE BAYES CLASSIFIER

In [283]:
X_train = df_final.loc[:2000, 'Body'].values
y_train = df_final.loc[:2000, 'sentiment'].values
X_test = df_final.loc[2000:, 'Body'].values
y_test = df_final.loc[2000:, 'sentiment'].values

In [284]:
#converting text into feature vectors with TF-IDF
vectorizer = TfidfVectorizer()
train_vectors = vectorizer.fit_transform(X_train)
test_vectors = vectorizer.transform(X_test)
print(train_vectors.shape, test_vectors.shape)

(1924, 12805) (1114, 12805)


In [285]:
clf = MultinomialNB().fit(train_vectors, y_train)

In [286]:
predicted = clf.predict(test_vectors)
print(accuracy_score(y_test,predicted))

0.0


XGBOOST

In [287]:
model = xgb.XGBClassifier(random_state=8888,max_depth=7, n_estimators=300, objective='binary:logistic')

In [288]:
model.fit(train_vectors, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=7, min_child_weight=1, missing=None,
       n_estimators=300, n_jobs=1, nthread=None,
       objective='multi:softprob', random_state=8888, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1)

In [289]:
prediction = model.predict(test_vectors)
print(accuracy_score(y_test,prediction))

0.05655296229802514
